# Homework 2 - EDA

In [2]:
# The regular imports

import pandas as pd
import numpy as np
import seaborn as sns


#### First, we do some data exploration to familiarize ourselves with the volume and kind of data

In [3]:
df = pd.read_csv("collegePlace.csv")
df

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,Male,Electronics And Communication,1,8,1,1,1
1,21,Female,Computer Science,0,7,1,1,1
2,22,Female,Information Technology,1,6,0,0,1
3,21,Male,Information Technology,0,8,0,1,1
4,22,Male,Mechanical,0,8,1,0,1
...,...,...,...,...,...,...,...,...
2961,23,Male,Information Technology,0,7,0,0,0
2962,23,Male,Mechanical,1,7,1,0,0
2963,22,Male,Information Technology,1,7,0,0,0
2964,22,Male,Computer Science,1,7,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966 entries, 0 to 2965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Age                2966 non-null   int64 
 1   Gender             2966 non-null   object
 2   Stream             2966 non-null   object
 3   Internships        2966 non-null   int64 
 4   CGPA               2966 non-null   int64 
 5   Hostel             2966 non-null   int64 
 6   HistoryOfBacklogs  2966 non-null   int64 
 7   PlacedOrNot        2966 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 185.5+ KB


In [5]:
df.Age.value_counts()

Age
21    1084
22     941
20     375
23     195
19     156
24     131
26      50
25      29
28       3
30       1
29       1
Name: count, dtype: int64

***
__Q1__ - What is the most common age for students in the dataset?

In [6]:
df.Age.mode()

0    21
Name: Age, dtype: int64

In [7]:
df['Age'].value_counts()

Age
21    1084
22     941
20     375
23     195
19     156
24     131
26      50
25      29
28       3
30       1
29       1
Name: count, dtype: int64

***
__Q2__ How many Female students are in the dataset?

In [8]:
df.Gender.value_counts()

Gender
Male      2475
Female     491
Name: count, dtype: int64

In [9]:
df[df['Gender']=='Female'].shape

(491, 8)

***
__Q3__ What percentage of the female student population lives in the hostel?

We calculate this by dividing the count of females living in the hostel by the total count of females in the dataset.

In other words, we calculate: 
Count of females living in the hostel / Total count of females

In [10]:
# Females living in the hostel
numerator = df[(df['Gender']=='Female') & (df['Hostel']==1)].shape[0]
numerator

230

In [11]:
denominator =  df[(df['Gender']=='Female')].shape[0]
denominator

491

In [12]:
numerator/denominator

0.4684317718940937

In [13]:
# Or we can do a quick cross-tab and do the calculations manually

pd.crosstab(df.Gender, df.Hostel, margins=True)

Hostel,0,1,All
Gender,,,
Female,261,230,491
Male,1907,568,2475
All,2168,798,2966


In [14]:
# Or we can do a quick cross-tab and normalize over rows using normalize='index'.
# This gives us the answer in the first row, second column

pd.crosstab(df.Gender, df.Hostel, margins=True, normalize='index')

Hostel,0,1
Gender,,
Female,0.531568,0.468432
Male,0.770505,0.229495
All,0.730951,0.269049


In [15]:
230/491

0.4684317718940937

In [16]:
# or we can do a quick pivot table 

df.pivot_table(index = ['Gender'],
               columns = ['Hostel'],
               values = ['Stream'],
               aggfunc = 'count', margins=True)

Stream           
Hostel      0    1   All
Gender                  
Female    261  230   491
Male     1907  568  2475
All      2168  798  2966

***
__Q4__ What is the correlation between the number of internships and the student's CGPA?

In [17]:
# We can use the correlation matrix

df.corr(numeric_only = True)

,Age,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
Age,1.000000,0.006552,-0.119787,0.003042,-0.042586,0.046943
Internships,0.006552,1.000000,0.023496,0.004617,-0.015118,0.179334
CGPA,-0.119787,0.023496,1.000000,0.014991,0.002576,0.588648
Hostel,0.003042,0.004617,0.014991,1.000000,0.103506,-0.038182
HistoryOfBacklogs,-0.042586,-0.015118,0.002576,0.103506,1.000000,-0.022337
PlacedOrNot,0.046943,0.179334,0.588648,-0.038182,-0.022337,1.000000


In [18]:
# Alternatively:

np.corrcoef(df.Internships, df.CGPA)

array([[1.        , 0.02349569],
       [0.02349569, 1.        ]])

So the answer is 0.02349569.

***
__Q5__ Age appears to be highly correlated with the likelihood of placement. True/False?

To answer this question, we look at the correlation between age and placement.  

Looks like correlation is quite low at 0.05, and age and placement are not 'highly correlated'.

In [19]:
np.corrcoef(df.PlacedOrNot, df.Age)

array([[1.        , 0.04694251],
       [0.04694251, 1.        ]])

***
__Q6__ - Which stream of study has the highest placement rate (measured as a percentage) considering all students?

In [20]:
# First, we do a crosstab of the stream and placement.  We can also do a pivot table, groupby, or use 
# other techniques.
# Next, we calculate the placement rate for each stream.

x = pd.crosstab(df.Stream, df.PlacedOrNot)
x

PlacedOrNot,0,1
Stream,,
Civil,171,146
Computer Science,324,452
Electrical,153,181
Electronics And Communication,173,251
Information Technology,282,409
Mechanical,224,200


In [21]:
# We calculate the placement rate, and sort descending to find out the stream with the highest rate.
# Electronics and communication has the highest placement rate at 173/251 = 59.1981%

x['Placement Rate'] = x[1]/(x[0] + x[1])
x.sort_values(by = 'Placement Rate', ascending= False, inplace=True)
x

PlacedOrNot,0,1,Placement Rate
Stream,,,
Electronics And Communication,173,251,0.591981
Information Technology,282,409,0.591896
Computer Science,324,452,0.582474
Electrical,153,181,0.541916
Mechanical,224,200,0.471698
Civil,171,146,0.460568


In [22]:
# If we need to programmatically pick the first item...

x.index[0]

'Electronics And Communication'

***
__Q7__ - Which stream of study has the highest placement rate (measured as a percentage) for females?

In [23]:
# This question is the same as the previous one except that we have to filter our dataframe first to include only females.
# Then we do all the same steps as before.

df_females = df[df['Gender']=='Female']
x = pd.crosstab(df_females.Stream, df_females.PlacedOrNot)
x['Placement Rate'] = x[1]/(x[0] + x[1])
x.sort_values(by = 'Placement Rate', ascending= False, inplace=True)
x

PlacedOrNot,0,1,Placement Rate
Stream,,,
Electronics And Communication,28,49,0.636364
Information Technology,57,81,0.586957
Electrical,21,29,0.580000
Computer Science,67,91,0.575949
Civil,17,12,0.413793
Mechanical,26,13,0.333333


In [24]:

x.index[0]

'Electronics And Communication'

***
__Q8__ - What is the overall placement rate (%) for females in the dataset?


In [25]:
# This question is very similar to the previous question, and calculated in the same way.
# The last line for 'All' gives us the answer of 56.0081%

df_females = df[df['Gender']=='Female']
x = pd.crosstab(df_females.Stream, df_females.PlacedOrNot, margins=True)
x['Placement Rate'] = x[1]/x['All']
x

PlacedOrNot,0,1,All,Placement Rate
Stream,,,,
Civil,17,12,29,0.413793
Computer Science,67,91,158,0.575949
Electrical,21,29,50,0.580000
Electronics And Communication,28,49,77,0.636364
Information Technology,57,81,138,0.586957
Mechanical,26,13,39,0.333333
All,216,275,491,0.560081


***
__Q9__ - What is the mean CGPA score for male and female students?

In [26]:
df.groupby(['Gender']).agg({"CGPA": "mean", "Age": "mean", "Hostel": "mean"})

,CGPA,Age,Hostel
Gender,,,
Female,7.081466,21.549898,0.468432
Male,7.072323,21.473131,0.229495


***
__Q10__ - How many students have completed at least 1 internship?

In [27]:
len(df[df['Internships']>0])

1635

# End
***